# 간단 config 설정하기
- ## Doc2Vec 모델을 생성할 때 귀찮은 config 설정 한번에 날려버리깅
- 어떤 config가 좋은지 알아보기 위해서는 다양한 모델 설정으로 확인하는 것이 중요할 것이다.
- 따라서 for문을 사용해 config를 설정한 뒤 모델을 생성하는 함수를 추가하였다. 항상 코드는 재사용하는 건 안비밀
- GPU가 없기 때문에 for문을 사용했는데 더 좋은 방법이 있으면.... 좋겠다.
- 작성 일시: 2018-06-08
- 수정 일시: 2018-06-08
- 작성자: 부현경 (hyunkyung.boo@gmail.com)

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import logging
import time


# 문서와 문서 ID를 태깅
def TaggedDocumentation(corpus_df, tag_df):
    custom_taggedDocument = []
    corpus_list = []
    for corpus, tag in zip(corpus_df, tag_df):
        splited_corpus = corpus.split(", ")
        custom_taggedDocument.append(TaggedDocument(list(splited_corpus), tags=[tag]))
        # print(TaggedDocumentation)
    # print(TaggedDocumentation[3:7])
    return custom_taggedDocument


# 모델을 생성한다. config를 이용해 모델 세팅을 할 수 있음
def create_doc2vec_model(TaggedDocumentation, config):
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = Doc2Vec(**config)
    model.build_vocab(TaggedDocumentation)

    # epoch 설정 후 학습
    start = time.time()
    for epoch in range(model.epochs):
        model.train(TaggedDocumentation, total_examples=len(TaggedDocumentation), epochs=model.epochs)
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay

    end = time.time()
    print("During Time: {}".format(end - start))
    return model


# 생성된 모델 저장
# ex. save_path = 'model/Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'

def save_doc2vec_model(model, save_path):
    model.save(save_path)
    print(save_path + "에 저장 완료")


# 다른 parameter 수정시에는 코드를 일부 수정해야 한다.
# config에 따른 모델 생성 + 저장을 한꺼번에 한다.
def getModels(s_list, w_list, n_list, sp_list, e_list, mc_list, a_list):
    cnt = 0
    for s in size:
        for w in window:
            for n in negative:
                for sp in sample:
                    for e in epoch:
                        for mc in min_count:
                            for a in alpha:

                                config = {
                                    'dm': 0,  # PV-DBOW / default 1
                                    'dbow_words': 1,  # w2v simultaneous with DBOW d2v / default 0
                                    'window': w,  # distance between the predicted word and context words
                                    'vector_size': s,  # vector size
                                    'alpha': a,  # learning-rate
                                    'seed': 1234,
                                    'sample': sp,
                                    'min_count': mc,  # ignore with freq lower
                                    'min_alpha': a,  # min learning-rate
                                    'workers': 1,  # multi cpu
                                    'hs': 1,  # hierarchical softmax / default 0
                                    'negative': n,  # negative sampling / default 5
                                    'epochs': e,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수. 또한 모델 학습시 반복 횟수 만큼 반복된다!!!
                                }

                                cnt += 1
                                label = "d{0}_w{1}_ng{2}_sp{3}_e{4}_mc{5}_a{6}".format(str(s), str(w), str(n), str(sp),
                                                                                       str(e), str(mc), str(a))
                                name = "{0}_D2V_setting_{1}.model".format(cnt, label)
                                model = create_doc2vec_model(TaggedDocuments, config)
                                save_root = 'D:\\'
                                doc2vec_save_path = save_root + name
                                save_doc2vec_model(model, doc2vec_save_path)
                                print("저장 완료! ", "저장 위치:", doc2vec_save_path)

In [ ]:
# 태그는 idx를 이용하며 TaggedDocumentation()를 이용해 문서와 태그를 매핑시켜준다.
TaggedDocuments = TaggedDocumentation(df['tokenized_user_review'], df['idx'])
print(TaggedDocuments[10:11])
# print(len(TaggedDocuments))


# getModel 함수에 사용되는 Parameter
# 8 * 4 * 5 * 4 * 3 * 7 * 2 = 26880
# 생성하는 시간도 무시하지 못한다...ㅜ^ㅜ
size = [5, 10, 15, 20, 40, 60, 100, 200]
window = [1, 3, 5, 8]
negative = [5, 10, 15, 20, 25]
sample = [0, 1e-2, 1e-4, 1e-5]
epoch = [3, 5, 10]
min_count = [1, 2, 3, 4, 5, 10, 20]
alpha = [0.015, 0.025]

# 모델을 생성해보자. # 아마 이번 코드에서는 TaggedDocuments가 없기 때문에 작동하지 않을테니 커스텀마이징을 해야한다!
getModels(size, window, negative, sample, epoch, min_count, alpha)